In [2]:
%matplotlib inline

import os
from helpers import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.3.8
SimpleITK version = 2.3.0


In [10]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
print(f'project folder = {BASE_DIR}')

raw_examples = [
    'fsl-open-dev_sub-001_T1w.nii.gz',
    'wash-120_sub-001_T1w.nii.gz',
    'kf-panda_sub-01_ses-3T_T1w.nii.gz',
    'listen-task_sub-UTS01_ses-1_T1w.nii.gz'
]

project folder = e:\ISEF\VascuIAR


In [5]:
raw_img_path = os.path.join(BASE_DIR, 'src', 'assets', 'raw_examples', raw_examples[0])
raw_img_ants = ants.image_read(raw_img_path, reorient='IAL') 

print(f'shape = {raw_img_ants.numpy().shape} -> (Z, X, Y)')

explore_3D_array(
    arr=raw_img_ants.numpy(),
    cmap='viridis'
)

shape = (192, 192, 160) -> (Z, X, Y)


interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

In [6]:
raw_img_ants = ants.image_read("assets/raw_examples/la_010.nii", reorient='IAL') 
print(f'shape = {raw_img_ants.numpy().shape} -> (Z, X, Y)')

explore_3D_array(
    arr=raw_img_ants.numpy(),
    cmap='viridis'
)

shape = (120, 320, 320) -> (Z, X, Y)


interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [7]:
# Denoise
transformed = ants.denoise_image(raw_img_ants, shrink_factor=8)

explore_3D_array_comparison(
    arr_before=raw_img_ants.numpy(),
    arr_after=transformed.numpy(),
    cmap='viridis'
)

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [12]:
# Morphological operations

"""
operation to apply
    "close" Morphological closing
    "dilate" Morphological dilation
    "erode" Morphological erosion
    "open" Morphological opening
"""

transformed = ants.morphology(raw_img_ants, radius=1, operation='erode', mtype='grayscale')

explore_3D_array_comparison(
    arr_before=raw_img_ants.numpy(),
    arr_after=transformed.numpy(),
    cmap='viridis'
)

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [3]:
raw_img_sitk = sitk.ReadImage("assets/raw_examples/la_010.nii", sitk.sitkFloat32)
raw_img_sitk = sitk.DICOMOrient(raw_img_sitk,'RPS')

print(f'shape = {sitk.GetArrayFromImage(raw_img_sitk).shape} -> (Z, X, Y)')
explore_3D_array(
    arr=sitk.GetArrayFromImage(raw_img_sitk),
    cmap='viridis'
)

shape = (120, 320, 320) -> (Z, X, Y)


interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [15]:
[ 3 ] * raw_img_sitk.GetDimension() # shrink 3 times

[3, 3, 3]

In [18]:
shrinkFactor = 3
transformed = sitk.Shrink( raw_img_sitk, [ shrinkFactor ] * raw_img_sitk.GetDimension() )

print(f'shape before = {sitk.GetArrayFromImage(raw_img_sitk).shape}')
print(f'shape after = {sitk.GetArrayFromImage(transformed).shape}')

explore_3D_array(sitk.GetArrayFromImage(transformed), cmap='gray')

shape before = (120, 320, 320)
shape after = (40, 106, 106)


interactive(children=(IntSlider(value=19, description='SLICE', max=39), Output()), _dom_classes=('widget-inter…

**Cropping**

- Cropping takes the orientation of the pixels for the reference of lower & upper boundaries vectors
- Pixel orientation = RPS = (left-to-Right, anterior-to-Posterior, inferior-to-Superior)

In [41]:
# crop nothing
# transformed = sitk.Crop(raw_img_sitk)
# transformed = sitk.Crop(raw_img_sitk, (0,0,0), (0,0,0))

# crop 40 from left to right             X,Y,Z
# transformed = sitk.Crop(raw_img_sitk, (40,0,0), (0,0,0))

# crop 40 from left to right, crop 30 from anterior to posterior
transformed = sitk.Crop(raw_img_sitk, (70,80,0), (100,100,0))

# crop 20 from left to right, crop 30 from anterior to posterior, 
# crop 10 from right to left, crop 5 from posterior to anterior. 
# transformed = sitk.Crop(raw_img_sitk, (20,30,0), (10,5,0)) 

# crop 40 from inferior to superior, crop 50 from superior to inferior
# transformed = sitk.Crop(raw_img_sitk, (0,0,40), (0,0,50)) 


print(f'shape before = {sitk.GetArrayFromImage(raw_img_sitk).shape}')
print(f'shape after = {sitk.GetArrayFromImage(transformed).shape}')

explore_3D_array(sitk.GetArrayFromImage(transformed))

shape before = (120, 320, 320)
shape after = (120, 140, 150)


interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [65]:
transformed = sitk.Crop(raw_img_sitk, (70,80,0), (100,100,0))
transformed = sitk.MomentsThreshold(transformed, 0, 1)

explore_3D_array(
    sitk.GetArrayFromImage(transformed)
)

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [71]:
transformed = sitk.Crop(raw_img_sitk, (70,80,0), (100,100,0))

explore_3D_array(
    sitk.GetArrayFromImage(transformed),
    cmap="nipy_spectral"
)

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [47]:
X = sitk.GetArrayFromImage(raw_img_sitk)
print(type(X))

<class 'numpy.ndarray'>


In [43]:
min_constant = int(sitk.GetArrayFromImage(raw_img_sitk).min())
max_constant = int(sitk.GetArrayFromImage(raw_img_sitk).max())
min_constant, max_constant

(0, 1907)

In [49]:
# Padding (as Cropping) takes the orientation of the pixels for the reference of lower & upper boundaries vectors
# Pixel orientation = RPS = (left-to-Right, anterior-to-Posterior, inferior-to-Superior)

# pad nothing
#transformed = sitk.ConstantPad(raw_img_sitk)
#transformed = sitk.ConstantPad(raw_img_sitk,(0,0,0),(0,0,0), constant)

# pad 10 from left to right
#transformed = sitk.ConstantPad(raw_img_sitk,(10,0,0),(0,0,0),constant)

# pad 10 from left to right, pad 15 from anterior to posterior
#transformed = sitk.ConstantPad(raw_img_sitk,(10,15,0),(0,0,0),constant)

# pad 10 from left to right, pad 15 from anterior to posterior, 
# pad 5 from right to left, pad 8 from posterior to anterior. 
transformed = sitk.ConstantPad(raw_img_sitk,(10,15,0),(5,8,0), max_constant) # fill padding with the max_constant pixel value


print(f'shape before = {sitk.GetArrayFromImage(raw_img_sitk).shape}')
print(f'shape after = {sitk.GetArrayFromImage(transformed).shape}')

explore_3D_array(sitk.GetArrayFromImage(transformed), cmap='viridis')

shape before = (120, 320, 320)
shape after = (120, 343, 335)


interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [50]:
# Denoise

transformed = sitk.CurvatureFlow(raw_img_sitk)

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(transformed),
    cmap='viridis'
)

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [67]:
"""
sitk.GrayscaleMorphologicalClosing
sitk.GrayscaleDilate
sitk.GrayscaleErode
sitk.GrayscaleMorphologicalOpening

sitk.BinaryMorphologicalClosing
sitk.BinaryDilate
sitk.BinaryErode
sitk.BinaryMorphologicalOpening
"""

transformed = sitk.GrayscaleMorphologicalOpening(raw_img_sitk)

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(transformed),
    cmap='nipy_spectral'
)

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [ ]:
transformed = sitk.DiscreteGaussian(raw_img_sitk)

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(transformed),
    cmap='viridis'
)

In [63]:
"""
sitk.OtsuThreshold
sitk.LiThreshold
sitk.TriangleThreshold
sitk.MomentsThreshold
"""

transformed = sitk.MomentsThreshold(raw_img_sitk, 0, 1)

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(transformed)
)

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [66]:
stats = sitk.StatisticsImageFilter()
stats.Execute(raw_img_sitk)


print('\tRaw img')
print("min  =", stats.GetMinimum())
print("max  =", stats.GetMaximum())
print("mean =", stats.GetMean())

	Raw img
min  = 0.0
max  = 1907.0
mean = 167.14945483398438


## Heed Mask

In [6]:
transformed = sitk.RescaleIntensity(raw_img_sitk, 0, 255)

#transformed = sitk.TriangleThreshold(transformed, 0, 1)
transformed = sitk.LiThreshold(transformed,0,1)

head_mask = transformed

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(head_mask),
)

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [7]:
shrinkFactor = 4
inputImage = raw_img_sitk

inputImage = sitk.Shrink( raw_img_sitk, [ shrinkFactor ] * inputImage.GetDimension() )
maskImage = sitk.Shrink( head_mask, [ shrinkFactor ] * inputImage.GetDimension() )

bias_corrector = sitk.N4BiasFieldCorrectionImageFilter()

corrected = bias_corrector.Execute(inputImage, maskImage)

In [8]:
log_bias_field = bias_corrector.GetLogBiasFieldAsImage(raw_img_sitk)
corrected_image_full_resolution = raw_img_sitk / sitk.Exp( log_bias_field )

explore_3D_array_comparison(
    sitk.GetArrayFromImage(raw_img_sitk),
    sitk.GetArrayFromImage(corrected_image_full_resolution), 
    cmap='nipy_spectral')

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [9]:
temp = sitk.Exp(log_bias_field)
temp = sitk.Mask(temp, head_mask)
explore_3D_array(sitk.GetArrayFromImage(temp), cmap='gray')

interactive(children=(IntSlider(value=59, description='SLICE', max=119), Output()), _dom_classes=('widget-inte…

In [12]:
raw_example = raw_examples[0]
raw_img_path = os.path.join(BASE_DIR, 'src', 'assets', 'raw_examples', raw_example)
# Save imgs
out_folder =  os.path.join(BASE_DIR, 'src', 'assets', 'preprocessed')
out_folder = os.path.join(out_folder, raw_example.split('.')[0]) # create folder with name of the raw file
os.makedirs(out_folder, exist_ok=True) # create folder if not exists

out_filename = add_suffix_to_filename(raw_example, suffix='biasFieldCorrected')
out_path = os.path.join(out_folder, out_filename)

print(raw_img_path[len(BASE_DIR):])
print(out_path[len(BASE_DIR):])

\src\assets\raw_examples\fsl-open-dev_sub-001_T1w.nii.gz
\src\assets\preprocessed\fsl-open-dev_sub-001_T1w\fsl-open-dev_sub-001_T1w_biasFieldCorrected.nii.gz


In [15]:
sitk.WriteImage(corrected_image_full_resolution, "assets/preprocessed/bias_corrected.nii")